# CHATBOT-WITH-LLAMA-2

## 1. Installing Necessary Libraries

In [1]:
# Cập nhật pip trước khi cài đặt
!pip install -U pip  -q

# Cài đặt bitsandbytes phiên bản ổn định
# !pip install bitsandbytes==0.39.0 --index-url https://jllllll.github.io/bitsandbytes-wheels/cu121/ -q

# Cài đặt phiên bản PyTorch tương thích
# !pip install -qqq torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  -q
!pip install torch -q

# Cài đặt transformers, peft và accelerate mới nhất
!pip install -qqq -U git+https://github.com/huggingface/transformers.git  -q
!pip install -qqq -U git+https://github.com/huggingface/peft.git  -q
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git  -q

# Cài đặt các thư viện hỗ trợ
!pip install -qqq datasets==2.12.0  -q
!pip install -qqq loralib==0.1.1  -q
!pip install -qqq einops==0.6.1  -q
!pip install -qqq wandb  -q
!pip install bitsandbytes==0.41.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 88.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 46.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 83.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 31.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 114.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 134.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [nvidia-cusolver-cu12]dia-cusolver-cu12]
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements t

In [2]:
!pip install bitsandbytes --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 105.9 MB/s eta 0:00:0000:0100:01


## 2. Importing Necessary Libraries

In [3]:
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split
import bitsandbytes as bnb
from datasets import load_dataset
from functools import partial
import os
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, AutoPeftModelForCausalLM
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, set_seed, BitsAndBytesConfig, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset

2025-05-24 05:26:38.545150: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748064398.752475      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748064398.814564      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "0"

In [5]:
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 3: Importing Llama 2 Chat HF Model

In [6]:
MODEL_NAME = "vilm/vinallama-2.7b-chat"

## 4. Performing Quantization

In [7]:
def create_bnb_config():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
    )

    return bnb_config

In [8]:
bnb_config = create_bnb_config()

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map={"": "cuda:0"},  # Chỉ sử dụng GPU 0
    trust_remote_code=True,
    use_safetensors=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = prepare_model_for_kbit_training(model)
model.gradient_checkpointing_enable()


config.json:   0%|          | 0.00/682 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.67M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/558 [00:00<?, ?B/s]

In [9]:
df = pd.read_csv("/kaggle/input/qa-dataset/vinmec_qa.csv")
# df = df.head(10000)
df.head()

,question,answer,context
0,liều dùng và cách sử dụng loperamide như thế nào?,Trẻ em: liều dùng tùy theo độ tuổi và tình trạ...,Liều và cách dùng của loperamide là người lớn:...
1,loperamide nên dùng bao nhiêu và dùng như thế ...,Trẻ em: liều dùng tùy theo độ tuổi và tình trạ...,Liều và cách dùng của loperamide là người lớn:...
2,những điều cần chú ý khi sử dụng loperamide là...,"Ngừng thuốc khi không hiệu quả trong 48 giờ, k...",Liều và cách dùng của loperamide là người lớn:...
3,có điều gì cần lưu ý khi dùng loperamide?,"Ngừng thuốc khi không hiệu quả trong 48 giờ, k...",Liều và cách dùng của loperamide là người lớn:...
4,loperamide được chỉ định trong trường hợp nào?,"Tổn thương gan, tắc ruột, táo bón, viêm loét đ...",Liều và cách dùng của loperamide là người lớn:...


In [10]:
df.shape

(10372, 3)

In [11]:
# Data Preparation
questions = df['question'].tolist()
contexts = df['context'].tolist()

# Đảm bảo tất cả các phần tử trong danh sách là chuỗi
questions = [str(question) for question in questions]
contexts = [str(context) for context in contexts]

In [12]:
import re

# # Hàm tìm từ ghép có dấu '_'
# def find_compound_words(texts):
#     compound_words = set()
#     for text in texts:
#         # Tìm các từ chứa dấu '_'
#         matches = re.findall(r'\b\w+_\w+\b', text)
#         compound_words.update(matches)
#     return list(compound_words)

# # Tìm từ ghép trong titles và detailed_contents
# compound_words_questions = find_compound_words(questions)
# compound_words_contexts = find_compound_words(contexts)

# # Kết hợp từ ghép từ cả titles và detailed_contents và đảm bảo tính duy nhất
# compound_words = list(set(compound_words_questions + compound_words_contexts))

# # Kiểm tra số lượng từ ghép đã tìm thấy và các từ là duy nhất
# print(f"Number of unique compound words: {len(compound_words)}")
# print(compound_words[:10])  # In ra 10 từ ghép đầu tiên để kiểm tra

In [13]:
# num_added_tokens = tokenizer.add_tokens(compound_words)

# model.resize_token_embeddings(len(tokenizer))

# print(f"Added {num_added_tokens} tokens. New vocab size: {len(tokenizer)}")

In [14]:
def find_words_not_in_vocab(texts, tokenizer_vocab):
    words_not_in_vocab = set()
    for text in texts:
        words = re.findall(r'\b\w+\b', text)
        for word in words:
            if word not in tokenizer_vocab:  # Kiểm tra xem từ có trong vocab không
                words_not_in_vocab.add(word)
    return list(words_not_in_vocab)

vocab = tokenizer.get_vocab()

words_not_in_question_vocab = find_words_not_in_vocab(questions, vocab)
words_not_in_context_vocab = find_words_not_in_vocab(contexts, vocab)

# Kết hợp các từ không có trong từ điển từ cả titles và detailed_contents
words_not_in_vocab = list(set(words_not_in_question_vocab + words_not_in_context_vocab))

print(f"Number of words not in vocab: {len(words_not_in_vocab)}")
print(words_not_in_vocab[:10])  # In ra 10 từ đầu tiên để kiểm tra


# # Thêm các từ mới vào tokenizer
num_added_tokens = tokenizer.add_tokens(words_not_in_vocab)

# # Cập nhật số lượng token trong mô hình để khớp với tokenizer mới
model.resize_token_embeddings(len(tokenizer))

# # In ra số lượng token đã thêm và kích thước từ điển mới
print(f"Added {num_added_tokens} tokens. New vocab size: {len(tokenizer)}")

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Number of words not in vocab: 4109
['nocardia', 'iodoquinol', 'ngọng', 'vôi', 'epstein', 'alanine', 'độn', 'menopur', 'uva', 'acei']


The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Added 4109 tokens. New vocab size: 50415


In [15]:
# Các token đặc biệt cần thêm
special_tokens = ["<|im_start|>", "<|im_end|>", "<|system|>", "<|user|>", "<|assistant|>", "system", "context", "user", "assistant"]

# Thêm vào tokenizer
tokenizer.add_tokens(special_tokens, special_tokens=True)

# Đồng thời resize embedding của mô hình nếu cần:
model.resize_token_embeddings(len(tokenizer))

Embedding(50419, 2560, padding_idx=0)

# test_sentence = "rào_cản tự_nhiên chống lại nhiễm_trùng da"
tokens = tokenizer.tokenize(test_sentence)
print(tokens)

In [16]:
# Split the data into train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.25, random_state=42)

In [17]:
def create_prompt_formats(sample):
    """
    Định dạng dữ liệu từ mẫu ('question', 'answer', 'context') theo định dạng ChatML
    :param sample: Từ điển mẫu chứa dữ liệu (bao gồm 'context' nếu có)
    """

    SYSTEM_PROMPT = "<|im_start|>system\nBạn là một trợ lí AI hữu ích. Hãy trả lời người dùng một cách chính xác.\n<|im_end>"
    CONTEXT_PROMPT = "<|im_start|>context\n{context}\n<|im_end>"  # Phần context mới
    USER_PROMPT = "<|im_start|>user\n{question}\n<|im_end>"
    ASSISTANT_PROMPT = "<|im_start|>assistant\n{answer}\n<|im_end>"
    
    # Định dạng từng phần
    system_part = SYSTEM_PROMPT
    context_part = CONTEXT_PROMPT.format(context=sample['context']) if 'context' in sample else ""
    user_part = USER_PROMPT.format(question=sample['question']) if 'question' in sample else ""
    assistant_part = ASSISTANT_PROMPT.format(answer=sample['answer']) if 'answer' in sample else ""
    
    # Nối các phần bằng hai ký tự xuống dòng, bỏ qua None
    parts = [part for part in [system_part, context_part, user_part, assistant_part] if part]
    formatted_prompt = "\n\n".join(parts)
    
    # Thêm trường 'text' vào sample
    sample["text"] = formatted_prompt

    return sample

In [18]:
def mask_labels_from_text(input_ids, labels, tokenizer, assistant_marker="<|im_start|>assistant\n"):
    # Tokenize marker để tìm marker token dưới dạng token ID
    marker_token_ids = tokenizer(assistant_marker, add_special_tokens=False)['input_ids']
    marker_len = len(marker_token_ids)

    # Tìm vị trí marker trong input_ids
    for i in range(len(input_ids) - marker_len + 1):
        if input_ids[i:i + marker_len] == marker_token_ids:
            start_index = i + marker_len
            break
    else:
        print("Không tìm thấy marker")
        return [-100] * len(labels)

    # Gán -100 cho phần trước marker
    return [-100 if i < start_index else labels[i] for i in range(len(labels))]


In [19]:
from datasets import Dataset


In [20]:
def preprocess_dataset(tokenizer, max_length, seed, dataset, stride=256):
    # Format lại input từ raw data
    dataset = dataset.map(create_prompt_formats)

    def tokenize_and_mask(examples):
        input_ids_all = []
        attention_mask_all = []
        labels_all = []

        for text in examples["text"]:
            tokenized = tokenizer(
                text,
                max_length=max_length,
                stride=stride,
                truncation=True,
                padding="max_length",
                return_overflowing_tokens=True,
                return_attention_mask=True,
            )

            for i in range(len(tokenized["input_ids"])):
                input_ids = tokenized["input_ids"][i]
                attention_mask = tokenized["attention_mask"][i]
                labels = input_ids.copy()

                # Áp dụng mask chính xác
                labels = mask_labels_from_text(input_ids, labels, tokenizer)

                input_ids_all.append(input_ids)
                attention_mask_all.append(attention_mask)
                labels_all.append(labels)

        return {
            "input_ids": input_ids_all,
            "attention_mask": attention_mask_all,
            "labels": labels_all,
        }

    tokenized_dataset = dataset.map(
        tokenize_and_mask,
        batched=True,
        remove_columns=dataset.column_names,
        desc="Tokenizing and masking",
    )

    tokenized_dataset = tokenized_dataset.shuffle(seed=seed)

    return tokenized_dataset


In [22]:
def create_peft_config(modules):
    """
    Create Parameter-Efficient Fine-Tuning config for your model
    :param modules: Names of the modules to apply Lora to
    """
    config = LoraConfig(
        r=32,  # dimension of the updated matrices
        lora_alpha=64,  # parameter for scaling
        target_modules=modules,
        lora_dropout=0.1,  # dropout probability for layers
        bias="none",
        task_type="CAUSAL_LM",
    )

    return config

In [23]:
def find_all_linear_names(model):
    #['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
    cls = bnb.nn.Linear4bit #if args.bits == 4 else (bnb.nn.Linear8bitLt if args.bits == 8 else torch.nn.Linear)
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:  # needed for 16-bit
        lora_module_names.remove('lm_head')
    return list(lora_module_names)

In [24]:
def print_trainable_parameters(model, use_4bit=False):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        num_params = param.numel()
        # if using DS Zero 3 and the weights are initialized empty
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params
    if use_4bit:
        trainable_params /= 2
    print(
        f"all params: {all_param:,d} || trainable params: {trainable_params:,d} || trainable%: {100 * trainable_params / all_param}"
    )

In [25]:
!pip install safetensors -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [26]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [ ]:
# Assuming you have your tokenizer and model loaded already
# max_length = get_max_length(model)  # Replace with actual function for getting max_length

# Apply preprocessing to datasets
train_dataset = preprocess_dataset(tokenizer, 1024, 42, train_dataset, stride=256)
val_dataset = preprocess_dataset(tokenizer, 1024, 42, val_dataset, stride=256)
test_dataset = preprocess_dataset(tokenizer, 1024, 42, test_dataset, stride=256)

In [28]:
from tqdm.auto import tqdm
import torch.nn.functional as nnf

In [29]:
from peft import PeftType

In [30]:
# Kiểm tra số lượng tham số trainable (nên rất nhỏ)
def print_trainable_parameters(model):
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"Trainable params: {trainable} / {total} ({100 * trainable / total:.2f}%)")

In [31]:
def train(model, tokenizer, train_dataset, val_dataset, output_dir, num_epochs=1):
    device = torch.device("cuda:0")
    model = model.to(device)

    checkpoint_dir = os.path.join(output_dir, "checkpoints")
    best_model_dir = os.path.join(output_dir, "best_model")
    os.makedirs(checkpoint_dir, exist_ok=True)
    os.makedirs(best_model_dir, exist_ok=True)
    os.makedirs(output_dir, exist_ok=True)

    torch.cuda.empty_cache()

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
        pad_to_multiple_of=8
    )

    # Optimize batch sizes based on available GPU memory - increase if possible
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=4,
        shuffle=True,
        collate_fn=data_collator,
        pin_memory=True,
        num_workers=4,
    )

    # Create DataLoader for validation
    val_dataloader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=4,
        shuffle=False,
        collate_fn=data_collator,
        pin_memory=True,
        num_workers=4,
    )

    start_epoch = 0
    best_loss = float('inf')

    # Allowlist PEFT classes for safe loading
    torch.serialization.add_safe_globals([LoraConfig, PeftType])

    # Enable PEFT config for model
    model.gradient_checkpointing_enable()
    model = prepare_model_for_kbit_training(model)
    modules = find_all_linear_names(model)
    peft_config = create_peft_config(modules)
    model = get_peft_model(model, peft_config)
    print_trainable_parameters(model)

    # Load checkpoint if available
    if os.path.exists(checkpoint_dir):
        checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith('checkpoint_epoch_')]
        if checkpoints:
            last_checkpoint = max(checkpoints, key=lambda x: int(x.split('_')[-1].split('.')[0]))
            checkpoint_path = os.path.join(checkpoint_dir, last_checkpoint)
            checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)

            model.load_state_dict(checkpoint['model_state_dict'], strict=False)
            start_epoch = checkpoint['epoch'] + 1
            best_loss = checkpoint.get('best_loss', float('inf'))
            print(f"Resuming from epoch {start_epoch} with best loss: {best_loss:.4f}")

    model = model.to(device)
    print_trainable_parameters(model)

    # Optimizer with higher learning rate and beta parameters adjusted for stability
    optimizer = torch.optim.AdamW(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=2e-4,
        weight_decay=0.01,
        betas=(0.9, 0.999),
        eps=1e-8
    )
    
    # Learning rate scheduler with warmup
    from transformers import get_linear_schedule_with_warmup
    num_training_steps = num_epochs * len(train_dataloader)
    num_warmup_steps = int(0.1 * num_training_steps)  # 10% warmup
    scheduler = get_linear_schedule_with_warmup(
        optimizer, 
        num_warmup_steps=num_warmup_steps, 
        num_training_steps=num_training_steps
    )

    # Enable mixed-precision training
    scaler = torch.cuda.amp.GradScaler(enabled=True)
    
    # Gradient accumulation steps - adjust based on GPU memory
    accumulation_steps = 2
    
    # Set to keep track of iterations for logging
    log_interval = 50
    
    for epoch in range(start_epoch, num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        step = 0
        optimizer.zero_grad(set_to_none=True)  # More efficient than zero_grad()

        for batch_idx, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")):
            # Move batch to device and use automatic mixed precision
            batch = {k: v.to(device, non_blocking=True) if torch.is_tensor(v) else v for k, v in batch.items()}
            
            # Use automatic mixed precision
            with torch.cuda.amp.autocast(enabled=True):
                outputs = model(**batch)
                loss = outputs.loss / accumulation_steps  # Normalize loss for gradient accumulation
            
            if torch.isnan(loss):
                print(f"Found NaN loss at batch {batch_idx}. Skipping step.")
                continue
            
            # Scale loss and backpropagate with mixed precision
            scaler.scale(loss).backward()
            running_loss += loss.item() * accumulation_steps
            step += 1
            
            if (step % accumulation_steps == 0) or (batch_idx == len(train_dataloader) - 1):
                # Unscale before gradient clipping
                scaler.unscale_(optimizer)
                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                # Update weights with scaled gradients
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad(set_to_none=True)
                scheduler.step()
                
                # Log loss periodically
                if batch_idx % log_interval == 0:
                    print(f"  Batch {batch_idx}/{len(train_dataloader)}: Loss = {loss.item() * accumulation_steps:.4f}")
                    
                # Free up memory
                if batch_idx % 10 == 0:
                    torch.cuda.empty_cache()

        avg_loss = running_loss / step

        # Validation phase - use limited number of batches to speed up
        model.eval()
        val_loss = 0.0
        val_step = 0
        val_max_batches = min(len(val_dataloader), 100)  # Limit validation batch count

        with torch.no_grad():
            for batch_idx, batch in enumerate(tqdm(val_dataloader, desc=f"Evaluating on Validation Set")):
                if batch_idx >= val_max_batches:
                    break
                    
                batch = {k: v.to(device, non_blocking=True) if torch.is_tensor(v) else v for k, v in batch.items()}
                
                with torch.cuda.amp.autocast(enabled=True):
                    outputs = model(**batch)
                    loss = outputs.loss

                val_loss += loss.item()
                val_step += 1

        avg_val_loss = val_loss / val_step

        print(f"Epoch {epoch+1}/{num_epochs}: Training loss: {avg_loss:.4f}, Validation loss: {avg_val_loss:.4f}")

        # Save the best model based on validation loss
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            print(f"New best model saved with validation loss: {best_loss:.4f}")
            # Save checkpoint and model if validation loss is improved
            checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),
                'loss': avg_loss,
                'best_loss': best_loss,
                'peft_config': model.peft_config if hasattr(model, 'peft_config') else peft_config,
            }
    
            checkpoint_path = os.path.join(checkpoint_dir, f'checkpoint_epoch_{epoch+1}.pt')
            try:
                torch.save(checkpoint, checkpoint_path, _use_new_zipfile_serialization=False)
                model.save_pretrained(output_dir)
                print(f"Epoch {epoch+1}/{num_epochs}: Checkpoint saved to {checkpoint_path}")
            except RuntimeError as e:
                print(f"Failed to save checkpoint: {e}")
                fallback_path = os.path.join('/tmp', f'checkpoint_epoch_{epoch+1}.pt')
                torch.save(checkpoint, fallback_path, _use_new_zipfile_serialization=False)
                print(f"Saved to fallback location: {fallback_path}")
        
        # Early stopping if no improvement for multiple epochs
        if hasattr(model, "_early_stopping_counter"):
            if avg_val_loss >= best_loss:
                model._early_stopping_counter += 1
                if model._early_stopping_counter >= 3:  # Stop after 3 epochs without improvement
                    print(f"Early stopping triggered after {epoch+1} epochs")
                    break
            else:
                model._early_stopping_counter = 0
        else:
            model._early_stopping_counter = 0

    tokenizer.save_pretrained(output_dir)
    print(f"Training completed. Final pretrained model saved to {output_dir}. Last checkpoint at epoch {epoch+1}.")

    torch.cuda.empty_cache()
    return model

In [32]:
# Specify the output directory
output_dir = "/kaggle/working/model/finetune"

# Train the model
model = train(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,  # Pass the training dataset
    val_dataset=val_dataset,      # Pass the validation dataset
    output_dir=output_dir,
    num_epochs=5
)

Trainable params: 50069504 / 1577158144 (3.17%)
Resuming from epoch 3 with best loss: 3.8946
Trainable params: 50069504 / 1577158144 (3.17%)


/tmp/ipykernel_35/2426473310.py:89: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=True)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadl

Epoch 4/5:   0%|          | 0/2126 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Evaluating on Validation Set:   0%|          | 0/402 [00:00<?, ?it/s]

...

Evaluating on Validation Set:   0%|          | 0/402 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Epoch 5/5: Training loss: 3.8822, Validation loss: 3.8709
New best model saved with validation loss: 3.8709


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:251: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


Epoch 5/5: Checkpoint saved to /kaggle/working/model/finetune/checkpoints/checkpoint_epoch_5.pt
Training completed. Final pretrained model saved to /kaggle/working/model/finetune. Last checkpoint at epoch 5.


In [33]:
# print(f"Model is on device: {next(model.parameters()).device}")

## 5. Import Necessary Functions

In [11]:
from peft import PeftModel, PeftConfig


In [12]:
# Bước 1: Load tokenizer
adapter_path = "/kaggle/working/model/finetune"
tokenizer = AutoTokenizer.from_pretrained(adapter_path)

# Bước 2: Load cấu hình adapter
peft_config = PeftConfig.from_pretrained(adapter_path)

# Bước 3: Load base model gốc từ Hugging Face (giống model bạn fine-tune)
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    return_dict=True,
    torch_dtype="auto"
)

base_model.resize_token_embeddings(len(tokenizer)) 

# Bước 4: Merge adapter vào base model
model = PeftModel.from_pretrained(base_model, adapter_path)

# (Tuỳ chọn) Nếu bạn muốn merge adapter vào full model để inference hiệu quả hơn:
model = model.merge_and_unload()

In [35]:
embedding_dim = model.config.hidden_size
max_length = model.config.max_position_embeddings

print(f"Embedding dim: {embedding_dim}")
print(f"Max length: {max_length}")

Embedding dim: 2560
Max length: 4096


In [13]:
def generate_answer(model, tokenizer, question, context=None, max_new_tokens=400, temperature=0.7, top_p=0.9, repetition_penalty=1.2):
    """
    Generate AI response using fine-tuned LLaMA model with optimized parameters for faster performance
    
    Parameters:
        model: Fine-tuned LLaMA model
        tokenizer: Model tokenizer
        question: User question (str)
        context: Optional context information (str) 
        max_new_tokens: Maximum tokens to generate
        temperature: Sampling temperature (0.7 = balanced creativity)
        top_p: Nucleus sampling threshold
        repetition_penalty: Penalty for repeated tokens
    
    Returns:
        str: AI assistant's response up to the first <|im_end> after assistant's content
    """
    # Prompt templates
    SYSTEM_PROMPT = """<|im_start|>system
Bạn là một trợ lí AI y tế hữu ích. Hãy trả lời người dùng một cách chính xác, ngắn gọn và chuyên nghiệp.
<|im_end|>"""

    CONTEXT_PROMPT = """<|im_start|>context
{context}
<|im_end|>"""

    USER_PROMPT = """<|im_start|>user
{question}
<|im_end|>"""

    ASSISTANT_PROMPT = """<|im_start|>assistant
"""

    # Build conversation prompt
    conversation = []
    conversation.append(SYSTEM_PROMPT)
    if context:
        conversation.append(CONTEXT_PROMPT.format(context=context))
    conversation.append(USER_PROMPT.format(question=question))
    conversation.append(ASSISTANT_PROMPT)
    prompt = "\n\n".join(conversation)

    # Prepare inputs
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=400  # Prevent context overflow
    ).to(device)

    # Generate response with faster parameters (greedy decoding and no beams)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,  # Use greedy decoding instead of sampling
            temperature=temperature,
            top_p=top_p,
            top_k=50,  # Limit vocab sampling to top 50 tokens (can be adjusted for speed)
            repetition_penalty=repetition_penalty,
            no_repeat_ngram_size=3,
            num_beams=1,  # Use greedy search for better speed
            early_stopping=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            bad_words_ids=[[tokenizer.unk_token_id]],  # Prevent unknown tokens
            length_penalty=1.0,  # Balanced length
            remove_invalid_values=True
        )

    # Process response
    response = tokenizer.decode(outputs[0], skip_special_tokens=False)
    
     # Find the position of the first <|im_end> after assistant's content
    start_pos = response.find('assistant') + len('assistant')
    end_pos = response.find('<|im_end>', start_pos)  # Find first <|im_end> after assistant

    if start_pos != -1 and end_pos != -1:
        response = response[start_pos:end_pos]  # Slice the content between assistant and <|im_end|>
    
    return response


In [37]:
# outputs = model.generate(
#             **inputs,
#             max_new_tokens=max_new_tokens,
#             do_sample=True,          # Bật sampling để câu trả lời đa dạng
#             temperature=temperature,
#             top_p=top_p,
#             repetition_penalty=repetition_penalty,  # Phạt các token lặp lại
#             no_repeat_ngram_size=2,  # Ngăn lặp lại cụm 2 từ
#         )

In [15]:
# Test hàm
question = "khi nào nên dùng loperamide?"
context = "Liều và cách dùng của loperamide là người lớn: khởi đầu 4 mg, sau đó mỗi lần tiêu chảy uống thêm 2 mg, tiêu chảy cấp: tối đa 5 ngày, liều thông thường: 6 - 8 mg/ngày, tối đa 16 mg/ngày, tiêu chảy do điều trị hóa chất ung thư: khởi đầu 4 mg, sau đó 2 mg mỗi 2 - 4 giờ cho tới khi hết tiêu chảy 12 giờ, tiêu chảy mạn: duy trì 4 - 8 mg/ngày, tối đa 16 mg, uống cho đến khi cầm tiêu chảy, trẻ em: liều dùng tùy theo độ tuổi và tình trạng bệnh. Chú ý khi sử dụng loperamide là ngừng thuốc khi không hiệu quả trong 48 giờ, khi bị táo bón, trướng bụng, liệt ruột, phụ nữ có thai: b3 (tga), b (fda). Chống chỉ định của loperamide là tổn thương gan, tắc ruột, táo bón, viêm loét đại tràng cấp, đau bụng không kèm tiêu chảy, bụng trướng, điều trị như liệu pháp khởi đầu trong hội chứng lỵ, viêm đại tràng giả mạc, tiêu chảy do vi khuẩn. Thận trọng khi sử dụng loperamide là suy giảm chức năng gan, viêm loét đại tràng, trẻ em dưới 6 tuổi. Chỉ định của loperamide là giảm thể tích chất thải sau thủ thuật mở thông hồi tràng hoặc đại tràng, phối hợp trong điều trị tiêu chảy cấp không có biến chứng ở người lớn. Nhóm thuốc – tác dụng của loperamide là thuốc điều trị tiêu chảy. Dạng bào chế - biệt dược của loperamide là viên nang - imodium, loperamide 2 mg. Tác dụng không mong muốn của loperamide là hiếm gặp: tắc ruột do liệt, quá mẫn, hội chứng stevens-johnson, hội chứng hoại tử biểu bì nhiễm độc, phản ứng phản vệ, thường gặp: táo bón, đau bụng, buồn nôn, ít gặp: mệt mỏi, chóng mặt, nhức đầu, trướng bụng, khô miệng, nôn."
answer = generate_answer(model, tokenizer, question, context)
# print("Câu hỏi:", question)
print("Câu trả lời:", answer)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Câu trả lời: <s><|im_start|>system 
Bạn là mộttrợlí AI ytếhữu ích. Hãy trảlờingười dùng một cáchchínhxác,ngắn gọn và chuyênnghiệp.
<|im_end|> 

<|im_start|>context 
Liều và cách dùngcủaloperamide làngườilớn:khởi đầu 4 mg, sauđó mỗi lần tiêuchảyuốngthêm 2 mg, tiêuchảy cấp: tối đa 5ngày,liềuthôngthường: 6 - 8 mg/ngày, tối đa 16 mg/ngày, tiêuchảy do điều trịhóa chất ungthư:khởi đầu 4 mg, sauđó 2 mg mỗi 2 - 4 giờ chotới khihết tiêuchảy 12 giờ, tiêuchảymạn:duytrì 4 - 8 mg/ngày, tối đa 16 mg,uống chođến khi cầm tiêuchảy, trẻ em:liều dùngtùy theo độtuổi và tìnhtrạng bệnh. Chú ý khi sửdụngloperamide làngừng thuốc khikhônghiệu quả trong 48 giờ, khi bịtáobón,trướngbụng,liệtruột, phụ nữ cóthai:b3 (tga), b (fda).Chống chỉ địnhcủaloperamide làtổnthương gan,tắcruột,táobón,viêmloét đạitràng cấp,đaubụngkhông kèm tiêuchảy,bụngtrướng, điều trị nhưliệuphápkhởi đầu trong hộichứnglỵ,viêm đạitràng giảmạc, tiêuchảy do vikhuẩn.Thậntrọng khi sửdụngloperamide làsuy giảmchứcnăng gan,viêmloét đạitràng, trẻ emdưới

In [39]:
!pip install nltk rouge-score -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'rouge-score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge-score'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [40]:
from nltk.translate.bleu_score import corpus_bleu
from rouge_score import rouge_scorer
import numpy as np

# Assuming you have a `generate_answer` function defined earlier
def calculate_bleu_and_rouge(model, tokenizer, test_df):
    # Prepare lists to store references (ground truth) and generated answers
    references = []
    predictions = []

    for idx, row in test_df.iterrows():
        question = row['question']
        context = row['context'] if 'context' in row else None  # Assuming the context is in the 'context' column
        ground_truth = row['answer']  # Assuming the answer column in test_df is named 'answer'
        
        # Generate AI response using the function
        generated_answer = generate_answer(model, tokenizer, question, context)
        
        # Append the reference and prediction to the respective lists
        references.append([ground_truth.split()])  # BLEU expects list of words (tokens)
        predictions.append(generated_answer.split())  # BLEU expects list of words (tokens)
        
    # Calculate BLEU score (corpus-level BLEU score)
    bleu_score = corpus_bleu(references, predictions)

    # Calculate ROUGE score
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    rouge_scores = {
        "rouge1": [],
        "rouge2": [],
        "rougeL": []
    }

    for ref, pred in zip(references, predictions):
        score = scorer.score(" ".join(ref[0]), " ".join(pred))
        rouge_scores["rouge1"].append(score["rouge1"].fmeasure)
        rouge_scores["rouge2"].append(score["rouge2"].fmeasure)
        rouge_scores["rougeL"].append(score["rougeL"].fmeasure)

    # Calculate average ROUGE scores
    avg_rouge1 = np.mean(rouge_scores["rouge1"])
    avg_rouge2 = np.mean(rouge_scores["rouge2"])
    avg_rougeL = np.mean(rouge_scores["rougeL"])

    # Output BLEU and ROUGE scores
    print(f"BLEU score: {bleu_score:.4f}")
    print(f"Average ROUGE-1: {avg_rouge1:.4f}")
    print(f"Average ROUGE-2: {avg_rouge2:.4f}")
    print(f"Average ROUGE-L: {avg_rougeL:.4f}")

# Example usage:
calculate_bleu_and_rouge(model, tokenizer, test_df[:50])

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are

BLEU score: 0.0174
Average ROUGE-1: 0.2241
Average ROUGE-2: 0.1242
Average ROUGE-L: 0.1660
